# Written Exercise 24

Design a method to determine whether an input is adversarial to a model that has been trained using an adversarial example augmented dataset (that is, the AugmentDefender from scratch defense). That is, the method will only have access to an instance that has not been used during training or defense, and has to determine whether it is adversarial or not. The method does not have the instance’s true label so you cannot simply test whether the defended model produces the correct label.


First, a threshold value will be drawn for the similarity measures as seen in written exercise 23. The following steps will be taken place to classify adversarial examples.


- Use the AugmentDefender from scratch defense to draw a threshold value.
- Randomly generate benign or adversarial examples and obtain similarity measure values at the input space.
- Classify each example and compare to the real value of whether the example is an benign one or an adversarial one. 


In [13]:
import hw4_part3
import numpy as np

import hw4_part1
import hw4_part2

import hw4_utils
import hw4_mnist

# Load Data
mnist_data = hw4_mnist.load_data()
train_X = mnist_data.train.X
train_Y = mnist_data.train.Y

m0 = hw4_part3.MNISTModelRegular()
m0.load()

### Build Data Augment From scratch model

In [14]:
m = hw4_part2.AugmentDefender(finetune = False, attacker = att, model = m0)
a_nft_XY = m.defend(train_X,train_Y,augment_ratio=0.3)

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]
                                                         
collecting attacks:   4%|▍         | 639/15000 [00:13<05:19, 45.01instances/s]


collecting attacks:   9%|▊         | 1279/15000 [00:27<05:26, 42.05instances/s]


collecting attacks:  13%|█▎        | 1905/15000 [00:40<05:02, 43.26instances/s]


collecting attacks:  17%|█▋        | 2542/15000 [00:54<04:23, 47.30instances/s]


collecting attacks:  21%|██▏       | 3210/15000 [01:08<04:15, 46.19instances/s]


...

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


collecting attacks:  94%|█████████▍| 14115/15000 [05:28<00:21, 41.34instances/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]


Train on 65000 samples
Epoch 1/2
65000/65000 [==============================] - 19s 293us/sample - loss: 0.4141 - acc: 0.9098
Epoch 2/2
65000/65000 [==============================] - 19s 288us/sample - loss: 0.1447 - acc: 0.9651


### Find the threshold for similarity measurement

For each number find the similarity measurement for not attacked and adversarial on input space.

In [16]:
integer = np.arange(10)
sample_size = 20
sub_train = 640
n = 10 # number of maximums to take

orig_sim = np.zeros(10)
adv_sim = np.zeros(10)
orig_idx = np.zeros(10)
adv_idx = np.zeros(10)

flattened_X = np.squeeze(
    np.reshape(train_X,[train_X.shape[0],train_X.shape[1]*train_X.shape[2],train_X.shape[3]])
    ,axis = 2)

# Loop through different integers
for i in integer:
    idx = np.where(train_Y[0:sub_train]==i)[0]
    
    # Attack 
    # CWL2 on M1
    att = hw4_part1.CWL2Attacker(
            model=m.model,
            num_steps=10,
            target=None,
            learning_rate=1.0,
            learning_rate_decay=0.9,
            k=0,
            c=4
        )
    
    att_X = att.attack(train_X[0:sub_train],train_Y[0:sub_train])
    att_X_flattened = np.squeeze(
        np.reshape(att_X,[att_X.shape[0],att_X.shape[1]*att_X.shape[2],att_X.shape[3]])
        ,axis = 2)

    
    # Loop through different instance of same integer
    for j in np.arange(sample_size):
        
        # Original
        rep = hw4_part3.Representer(m.model, np.expand_dims(train_X[idx[j]],axis=0)
                                    ,np.expand_dims(train_Y[idx[j]],axis=0))
        temp_input_sim = rep.similarity(train_X[0:1600])
        
        max_ind = (-temp_input_sim).argsort()[:n]
        
        for k in max_ind:
            if i == train_Y[k]:
                orig_sim[i] += 1/n
                
        # Adversarial
        rep = hw4_part3.Representer(m.model, np.expand_dims(att_X[idx[j]],axis=0)
                                    ,np.expand_dims(train_Y[idx[j]],axis=0))
        temp_input_sim = rep.similarity(train_X[0:1600])
        
        max_ind = (-temp_input_sim).argsort()[:n]
        
        for k in max_ind:
            if i == train_Y[k]:
                adv_sim[i] += 1/n
        
        orig_idx[i] += 1
        adv_idx[i] += 1
        
orig_th = np.divide(orig_sim,orig_idx)
adv_th = np.divide(adv_sim,adv_idx)


attacking:   0%|          | 0/40 [00:00<?, ?batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

attacking batch:  10%|█         | 1/10 [00:00<00:05,  1.53step/s]

                                                                 
attacking:   2%|▎         | 1/40 [00:01<00:56,  1.45s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   5%|▌         | 2/40 [00:01<00:42,  1.12s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   8%|▊         | 3/40 [00:02<00:33,  1.10batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  10%|█         | 4/40 [00:02<00:26,  1.36batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:

                                                                 
attacking:   2%|▎         | 1/40 [00:01<00:56,  1.46s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   5%|▌         | 2/40 [00:01<00:42,  1.12s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   8%|▊         | 3/40 [00:02<00:32,  1.13batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  10%|█         | 4/40 [00:02<00:25,  1.39batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:02<00:21,  1.66batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:03

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   5%|▌         | 2/40 [00:02<00:48,  1.28s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   8%|▊         | 3/40 [00:02<00:37,  1.01s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  10%|█         | 4/40 [00:02<00:29,  1.22batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:03<00:24,  1.46batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:03<00:20,  1.68batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                              

                                                         
attacking:   5%|▌         | 2/40 [00:02<00:54,  1.43s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:   8%|▊         | 3/40 [00:02<00:41,  1.13s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  10%|█         | 4/40 [00:03<00:33,  1.08batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:03<00:27,  1.29batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:04<00:22,  1.48batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  18%|█▊        | 7/40 [00:04<00:19, 

                                                         
attacking:   8%|▊         | 3/40 [00:03<00:46,  1.27s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  10%|█         | 4/40 [00:03<00:37,  1.03s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:04<00:30,  1.15batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:04<00:25,  1.32batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  18%|█▊        | 7/40 [00:05<00:22,  1.47batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  20%|██        | 8/40 [00:05<00:20, 

                                                         
attacking:  10%|█         | 4/40 [00:03<00:41,  1.15s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  12%|█▎        | 5/40 [00:04<00:34,  1.03batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:05<00:28,  1.19batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  18%|█▊        | 7/40 [00:05<00:24,  1.33batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  20%|██        | 8/40 [00:06<00:22,  1.45batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  22%|██▎       | 9/40 [00:06<00:20, 

                                                         
attacking:  12%|█▎        | 5/40 [00:04<00:37,  1.07s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  15%|█▌        | 6/40 [00:05<00:31,  1.08batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  18%|█▊        | 7/40 [00:06<00:27,  1.20batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  20%|██        | 8/40 [00:06<00:24,  1.31batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  22%|██▎       | 9/40 [00:07<00:22,  1.40batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  25%|██▌       | 10/40 [00:07<00:20,

                                                         
attacking:  15%|█▌        | 6/40 [00:06<00:34,  1.02s/batch]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  18%|█▊        | 7/40 [00:06<00:29,  1.10batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  20%|██        | 8/40 [00:07<00:26,  1.20batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  22%|██▎       | 9/40 [00:08<00:24,  1.29batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  25%|██▌       | 10/40 [00:08<00:22,  1.35batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  28%|██▊       | 11/40 [00:09<00:20

                                                         
attacking:  18%|█▊        | 7/40 [00:07<00:32,  1.02batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  20%|██        | 8/40 [00:07<00:29,  1.10batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  22%|██▎       | 9/40 [00:08<00:26,  1.18batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  25%|██▌       | 10/40 [00:09<00:24,  1.24batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  28%|██▊       | 11/40 [00:10<00:22,  1.29batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  30%|███       | 12/40 [00:10<00:2

                                                         
attacking:  20%|██        | 8/40 [00:08<00:30,  1.03batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  22%|██▎       | 9/40 [00:09<00:27,  1.11batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  25%|██▌       | 10/40 [00:10<00:25,  1.16batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  28%|██▊       | 11/40 [00:10<00:24,  1.21batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  30%|███       | 12/40 [00:11<00:22,  1.23batch/s]

attacking batch:   0%|          | 0/10 [00:00<?, ?step/s]

                                                         
attacking:  32%|███▎      | 13/40 [00:12<00:

In [17]:
orig_th

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [20]:
adv_th

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])